# Evolver Loop 9 Analysis

## Key Questions:
1. What are the best pre-optimized solutions in snapshots?
2. Can we find a solution that beats 70.676 (baseline)?
3. What techniques from top kernels should we implement?

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
import glob
import os

getcontext().prec = 30

print("Libraries loaded")

In [ ]:
# Define tree class and scoring functions
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(0), float(tip_y)),
            (float(top_w / 2), float(tier_1_y)),
            (float(top_w / 4), float(tier_1_y)),
            (float(mid_w / 2), float(tier_2_y)),
            (float(mid_w / 4), float(tier_2_y)),
            (float(base_w / 2), float(base_y)),
            (float(trunk_w / 2), float(base_y)),
            (float(trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(base_y)),
            (float(-base_w / 2), float(base_y)),
            (float(-mid_w / 4), float(tier_2_y)),
            (float(-mid_w / 2), float(tier_2_y)),
            (float(-top_w / 4), float(tier_1_y)),
            (float(-top_w / 2), float(tier_1_y)),
        ])

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return s[1:]
    return str(s)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    subset = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in subset.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def get_bounding_box_side(trees):
    if not trees:
        return 0
    all_coords = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    return max(x_range, y_range)

def has_overlap(trees, tolerance=1e-15):
    if len(trees) <= 1:
        return False, []
    polygons = [t.polygon for t in trees]
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > tolerance:
                    overlaps.append((i, j, intersection.area))
    return len(overlaps) > 0, overlaps

def score_submission(df):
    total = 0
    for n in range(1, 201):
        trees = load_trees_for_n(df, n)
        if len(trees) == n:
            side = get_bounding_box_side(trees)
            total += (side ** 2) / n
    return total

print("Functions defined")

In [ ]:
# Check the telegram solutions
telegram_files = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv',
]

for fp in telegram_files:
    if os.path.exists(fp):
        df = pd.read_csv(fp)
        if len(df) == 20100:
            score = score_submission(df)
            print(f"{fp.split('/')[-1]}: score = {score:.6f}")

In [ ]:
# Check our baseline
baseline_df = pd.read_csv('/home/code/experiments/000_baseline/submission.csv')
baseline_score = score_submission(baseline_df)
print(f"Baseline score: {baseline_score:.6f}")
print(f"Target: 68.887226")
print(f"Gap: {baseline_score - 68.887226:.6f}")

In [ ]:
# Find the BEST valid submission across all snapshots
# This time, we'll check for overlaps properly

print("Searching for best VALID submissions...")
print()

snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
all_csvs = glob.glob(f'{snapshot_dir}/**/*.csv', recursive=True)
print(f"Found {len(all_csvs)} CSV files")

best_valid = {'score': baseline_score, 'path': 'baseline', 'has_overlap': False}
best_any = {'score': baseline_score, 'path': 'baseline'}

processed = 0
for csv_path in all_csvs:
    try:
        df = pd.read_csv(csv_path)
        if 'id' not in df.columns or len(df) != 20100:
            continue
        if not df['id'].iloc[0].startswith('001_'):
            continue
            
        score = score_submission(df)
        
        if score < best_any['score']:
            best_any = {'score': score, 'path': csv_path}
        
        # Only check for overlaps if score is better than best valid
        if score < best_valid['score']:
            # Quick overlap check on a few N values
            has_ovlp = False
            for n in [2, 3, 5, 10, 20, 50, 100]:
                trees = load_trees_for_n(df, n)
                if len(trees) == n:
                    ovlp, _ = has_overlap(trees)
                    if ovlp:
                        has_ovlp = True
                        break
            
            if not has_ovlp:
                best_valid = {'score': score, 'path': csv_path, 'has_overlap': False}
                print(f"New best valid: {score:.6f} from {csv_path.split('/')[-1]}")
        
        processed += 1
        if processed % 500 == 0:
            print(f"Processed {processed}...")
    except:
        pass

print(f"\nProcessed {processed} valid submissions")
print(f"\nBest ANY score: {best_any['score']:.6f} from {best_any['path'].split('/')[-1]}")
print(f"Best VALID score: {best_valid['score']:.6f} from {best_valid['path'].split('/')[-1]}")

In [ ]:
# Let's check if the best valid submission has any overlaps at all
if best_valid['path'] != 'baseline':
    print(f"Checking {best_valid['path']} for overlaps...")
    df = pd.read_csv(best_valid['path'])
    
    overlap_ns = []
    for n in range(1, 201):
        trees = load_trees_for_n(df, n)
        if len(trees) == n:
            ovlp, details = has_overlap(trees)
            if ovlp:
                overlap_ns.append((n, details[0][2] if details else 0))
    
    if overlap_ns:
        print(f"Found overlaps in {len(overlap_ns)} N values:")
        for n, area in overlap_ns[:10]:
            print(f"  N={n}: overlap area = {area:.10f}")
    else:
        print("No overlaps found! This submission should be valid.")
else:
    print("Best valid is baseline - no need to check")

In [ ]:
# Summary
print("="*60)
print("SUMMARY")
print("="*60)
print(f"Baseline score: {baseline_score:.6f}")
print(f"Target score: 68.887226")
print(f"Gap to target: {baseline_score - 68.887226:.6f} ({(baseline_score - 68.887226) / 68.887226 * 100:.2f}%)")
print()
print(f"Best valid score found: {best_valid['score']:.6f}")
print(f"Best ANY score found: {best_any['score']:.6f}")
print()
print("Key insight: The gap is 1.79 points (2.6%). This is too large for micro-optimization.")
print("Need fundamentally different approach.")
print("="*60)